In [144]:
%pylab inline
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler

Populating the interactive namespace from numpy and matplotlib


In [145]:
# load train and test data
wnv_train = np.loadtxt('../working/train_f.csv', delimiter=',', dtype=float64, skiprows=1)

In [146]:
# split the data into training and validation sets
min_max_scaler = MinMaxScaler()
wnv_mm = min_max_scaler.fit_transform(wnv_train[:,2:])
X_train, X_test, y_train, y_test = train_test_split(wnv_mm, wnv_train[:,1].astype(int))

In [147]:
# parameters
n_neighbors = 3
n_components = 65
n_estimators = 500
n_jobs = 3

In [148]:
#clf = KNeighborsClassifier(n_neighbors=n_neighbors)
clf = ExtraTreesClassifier(n_estimators=n_estimators, n_jobs=n_jobs)
clf.fit(Xn_train, y_train)

# use the model to predict the labels of the test data
predicted = clf.predict(Xn_test)
expected = y_test

In [149]:
# accuracy
matches = (predicted == expected)
print matches.sum() / float(len(matches))

# f1 score
from sklearn import metrics
print metrics.classification_report(expected, predicted)

# confusion matrix
print metrics.confusion_matrix(expected, predicted)

0.909783022459
             precision    recall  f1-score   support

          0       0.94      0.97      0.95      2471
          1       0.03      0.02      0.02       156

avg / total       0.89      0.91      0.90      2627

[[2387   84]
 [ 153    3]]


In [150]:
print np.count_nonzero(predicted == 1)
print np.count_nonzero(y_test)

87
156
